In [17]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [18]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128

num_input = 784
num_classes = 10
dropout = 0.75

X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

In [19]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    
    x = tf.reshape(x, shape = [-1, 28, 28, 1])
    
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [20]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

logits = conv_net(X, weights, biases, dropout)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [21]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_op, feed_dict = {X:batch_x, Y:batch_y, keep_prob:dropout})
        if step % 100 == 0:
            loss, accu = sess.run([loss_op, accuracy], feed_dict = {X:batch_x, Y:batch_y, keep_prob:1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(accu))
    print("Training finished!")
    
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

Step 100, Minibatch Loss= 7631.1211, Training Accuracy= 0.719
Step 200, Minibatch Loss= 2586.0518, Training Accuracy= 0.852
Step 300, Minibatch Loss= 1982.1277, Training Accuracy= 0.898
Step 400, Minibatch Loss= 1231.5657, Training Accuracy= 0.891
Step 500, Minibatch Loss= 1620.0596, Training Accuracy= 0.891
Step 600, Minibatch Loss= 486.0205, Training Accuracy= 0.953
Step 700, Minibatch Loss= 1226.7910, Training Accuracy= 0.930
Step 800, Minibatch Loss= 232.1834, Training Accuracy= 0.961
Step 900, Minibatch Loss= 983.3777, Training Accuracy= 0.945
Step 1000, Minibatch Loss= 452.4937, Training Accuracy= 0.930
Step 1100, Minibatch Loss= 443.1634, Training Accuracy= 0.938
Step 1200, Minibatch Loss= 412.0157, Training Accuracy= 0.914
Step 1300, Minibatch Loss= 489.9909, Training Accuracy= 0.938
Step 1400, Minibatch Loss= 558.5195, Training Accuracy= 0.953
Step 1500, Minibatch Loss= 85.7023, Training Accuracy= 0.969
Step 1600, Minibatch Loss= 301.3355, Training Accuracy= 0.961
Step 1700, M